Getting familiar with all CO and PPH data

In [1]:
import numpy as np
import geopandas as gp
import shapely as sp
import matplotlib.pyplot as plt
import contextily as cx
import cartopy as cp
from datetime import datetime as dt
from datetime import timedelta
import xarray as xr
import pandas as pd
import re
import os
from utils_filter import *
from utils_datetime import *

In [2]:
raw_dir = '~/Downloads'
outlook_raw_location = raw_dir + '/outlooks'
report_raw_location = raw_dir + '/storm_reports'
pph_raw_location = raw_dir + '/pph'
outlook_save_location = 'data/outlooks'
report_save_location = 'data/storm_reports'
pph_save_location = 'data/pph'

# Read in Convective outlooks and filter moderate days

In [ ]:
# Read convective outlooks into outlooks
year_list = [[1987, 1991], [1992, 1999], [2000, 2007], [2008, 2015], [2016, 2023]]
for years, i in zip(year_list, range(len(year_list))):
    print('reading file ' + str(i) + ', years ' + str(years[0]) +'-' + str(years[1]))
    if i == 0:
        outlooks_original = gp.read_file(outlook_raw_location + '/outlooks_' + str(years[0]) + '01010000_' + str(years[1]) + '12312359')
    else:
        outlooks_original = outlooks_original.append(gp.read_file(outlook_raw_location + '/outlooks_' + str(years[0]) + '01010000_' + str(years[1]) + '12312359'))
print('files read')
    
outlooks_original



In [ ]:
# make dates datetime
outlooks = outlooks_original
outlooks['ISSUE'] = parse_datetime(outlooks['ISSUE'])
outlooks['EXPIRE'] = parse_datetime(outlooks['EXPIRE'])
outlooks['PRODISS'] = parse_datetime(outlooks['PRODISS'])

In [ ]:
# reset incicies
outlooks = outlooks.reset_index(drop=True)
outlooks

In [ ]:
outlooks = fix_month_issue(outlooks)    
outlooks

In [ ]:
# add column with just valid date
outlooks['DATE'] = create_dates(outlooks['EXPIRE'], -1)

# identify dates with MDT
mod_dates = identify_dates_above_threshold(outlooks, 'MDT')

In [ ]:
# Plot number of MDT days by year
years_of_mdt = get_years(mod_dates)
plt.hist(years_of_mdt, bins=range(min(years_of_mdt), max(years_of_mdt) + 1, 1))

In [ ]:
# dataframe containing only outlooks for days in which there was a MDT risk
mdt_outlooks = outlooks[outlooks['DATE'].isin(mod_dates)]

# convert datetimes back to strings
outlooks = revert_all_datetimes(outlooks)
mdt_outlooks = revert_all_datetimes(mdt_outlooks)

# save dataframes
outlooks.iloc[:int(len(outlooks)/2)].to_file(outlook_save_location + '/all_outlooks_1.shp')
outlooks.iloc[int(len(outlooks)/2)+1:].to_file(outlook_save_location + '/all_outlooks_2.shp')
mdt_outlooks.to_file(outlook_save_location + '/mdt_outlooks.shp')


# Now read, combine, filter (to mdt), and save PPH data

In [4]:
hazard_types = ['wind', 'sig_wind', 'hail', 'sig_hail', 'tor', 'sig_tor', 'all_svr', 'all_sig_svr']
for hazard, i in zip(hazard_types, range(len(hazard_types))):
    print('reading in ' + hazard + ' pph')
    if i == 0:
        pph_data = xr.open_dataset(pph_raw_location + '/pper_' + hazard + '_1979_2023.nc')
    else:
        new_data = xr.open_dataset(pph_raw_location + '/pper_' + hazard + '_1979_2023.nc')
        pph_data = xr.merge([pph_data, new_data])




reading in wind pph
reading in sig_wind pph
reading in hail pph


C:\Users\miles\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xarray\backends\plugins.py:71: RuntimeWarning: Engine 'gini' loading failed:
name 'basic' is not defined
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


reading in sig_hail pph
reading in tor pph
reading in sig_tor pph
reading in all_svr pph
reading in all_sig_svr pph


In [ ]:
# select pph data on days with mdt risk
pph_data_mod = pph_data.sel(time=pph_data.time.dt.date.isin(mod_dates.tolist()))

In [7]:
# save full and moderate pph datasets
pph_data.to_netcdf(pph_save_location + '/all_pph.nc')
pph_data_mod.to_netcdf(pph_save_location + '/mdt_pph.nc')

In [5]:
pph_data

<xarray.Dataset>
Dimensions:              (time: 16436, x: 93, y: 65)
Coordinates:
  * time                 (time) datetime64[ns] 1979-01-01T12:00:00 ... 2023-1...
  * x                    (x) float64 0.0 1.0 2.0 3.0 4.0 ... 89.0 90.0 91.0 92.0
  * y                    (y) float64 0.0 1.0 2.0 3.0 4.0 ... 61.0 62.0 63.0 64.0
Data variables:
    lat                  (y, x) float64 12.19 12.39 12.58 ... 57.68 57.49 57.29
    lon                  (y, x) float64 -133.5 -132.8 -132.1 ... -50.44 -49.39
    p_perfect_wind       (time, y, x) float64 ...
    p_perfect_sig_wind   (time, y, x) float64 ...
    p_perfect_hail       (time, y, x) float64 ...
    p_perfect_sig_hail   (time, y, x) float64 ...
    p_perfect_tor        (time, y, x) float64 ...
    p_perfect_sig_tor    (time, y, x) float64 ...
    p_perfect_totalsvr   (time, y, x) float64 ...
    p_perfect_totsigsvr  (time, y, x) float64 ...
Attributes:
    title:         Practically Perfect Wind Hindcasts
    grid:          80-km NCEP 211
    sigma:         1.5
    author:        Dr. Victor Gensini
    author_email:  vgensini@niu.edu
    citation:      https://doi.org/10.1175/BAMS-D-19-0321.1

# Read in, combine, filter, and save raw storm reports

In [ ]:
columns =['STATE', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_NAME', 'CZ_NAME', 'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE', 'MAGNITUDE', 'MAGNITUDE_TYPE', 'TOR_F_SCALE', 'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE', 'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE', 'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH', 'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON']
event_types = ['Funnel Cloud', 'Hail', 'Marine Hail', 'Marine Thunderstorm Wind', 'Thunderstorm Wind', 'Tornado', 'Waterspout']




In [ ]:
# read in reports and combine into all_reports
first = True
for file in os.listdir(os.fsencode(report_raw_location)):
    filename = os.fsdecode(file)
    if 'StormEvents_details-ftp_v1.0_d' in filename:
        print('reading' + filename)
        reports = gp.read_file(report_raw_location + '/' + filename)
        if first:
            all_reports = filter_reports(reports, columns, event_types)
            first = False
        else:
            all_reports = all_reports.append(filter_reports(reports, columns, event_types))


In [ ]:
# filter all_reports to get all_reports_mdt
all_reports['DATE'] = parse_datetime_reports(all_reports['BEGIN_DATE_TIME'])
all_reports_mdt = all_reports[all_reports['DATE'].isin(mod_dates.tolist())]



In [ ]:
# save report data
all_reports.to_csv(report_save_location + '/all_reports.csv')
all_reports_mdt.to_csv(report_save_location + '/mdt_reports.csv')
